In [1]:
from google.colab import files

uploaded_model1 = files.upload()





Saving model3a.csv to model3a (1).csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
from sklearn.decomposition import NMF
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Load the dataset
data = pd.read_csv('model3a.csv')

# Preprocess the data
# Convert labels to one-hot encoding
labels = data.iloc[:, 1:]
text_data = data['text']

# Train-test split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Matrix Factorization (NMF)
num_topics = 30
nmf = NMF(n_components=num_topics)
W = nmf.fit_transform(labels)
H = nmf.components_

# Neural Collaborative Filtering (NCF)
num_users = labels.shape[0]
num_items = labels.shape[1]
embedding_dim = 64

# Define NCF model
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(num_users, embedding_dim)(user_input)
item_embedding = Embedding(num_items, embedding_dim)(item_input)

user_flat = Flatten()(user_embedding)
item_flat = Flatten()(item_embedding)

concat = Concatenate()([user_flat, item_flat])

dense_1 = Dense(64, activation='relu')(concat)
dense_2 = Dense(32, activation='relu')(dense_1)

output = Dense(1, activation='sigmoid')(dense_2)

ncf_model = Model(inputs=[user_input, item_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train NCF model
user_indices = train_data.index
item_indices = range(num_items)
labels_train = labels.values

# Define positive and negative samples for training
def generate_samples():
    for user in user_indices:
        for item in item_indices:
            yield user, item, labels_train[user, item]

samples = np.array(list(generate_samples())).T
users_train = samples[0]
items_train = samples[1]
labels_train = samples[2]

ncf_model.fit([users_train, items_train], labels_train, epochs=5, batch_size=64)
# Save the trained model
#ncf_model.save("ncf_model.h5")

# Download the saved model file
#files.download("ncf_model.h5")

# Recommendation for a given text
def recommend_labels(text, n=5):
    # Predict labels using NCF model
    user_idx = np.array([0] * num_items)  # Assuming user index 0 for recommendation
    item_idx = np.array(range(num_items))
    scores = ncf_model.predict([user_idx, item_idx])

    # Sort labels by predicted scores and recommend the top n
    recommended_labels = np.argsort(-scores, axis=None)[:n]
    return recommended_labels

# Example: Recommend labels for a given text
text_user_input = "I made a goal to make come up with an official project in Crafts for Service.&#10;I felt inspired&#10;I talked to peers about possible ideas&#10;That doing these projects can be fairly easy&#10;I wouldn&apos;t do anything differently except plan things sooner"
recommended_labels = recommend_labels(text_user_input, n=20)

# Print recommended labels
print("Recommended Microcredentials for the input text:")
for label in recommended_labels:
    print(f"Label {label}: {labels.columns[label]}")


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Epoch 1/5
6593/6593 [==============================] - 40s 6ms/step - loss: 0.1182 - accuracy: 0.9651
Epoch 2/5
6593/6593 [==============================] - 40s 6ms/step - loss: 0.1006 - accuracy: 0.9690
Epoch 3/5
6593/6593 [==============================] - 39s 6ms/step - loss: 0.0885 - accuracy: 0.9722
Epoch 4/5
6593/6593 [==============================] - 39s 6ms/step - loss: 0.0796 - accuracy: 0.9747
Epoch 5/5
5/5 [==============================] - 0s 3ms/step
Recommended Microcredentials for the input text:
Label 0: 111
Label 130: 651
Label 18: 142
Label 28: 216
Label 21: 145
Label 131: 652
Label 122: 632
Label 9: 124
Label 1: 112
Label 132: 653
Label 42: 244
Label 22: 146
Label 45: 247
Label 46: 248
Label 68: 416
Label 36: 231
Label 29: 217
Label 123: 633
Label 3: 114
Label 133: 711
